# Homework 3

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch.nn
import torch.nn.functional
import torch.optim
import torch.utils.data

## Problem 1

Let's explore hyperparameter optimization.  We'll do something relatively simple that's actually not far from what most people do to study the effect of hyperparameters on their model.  Let's revisit the neural network we implemented to learn the $\sin$ function in our `NeuralNetworks2` notebook.  

In [ ]:
torch.manual_seed(123)
x_train = 6 * torch.rand((1000, 1)) - 3
y_train = torch.sin(x_train)
x_test = 6 * torch.rand((100, 1)) - 3
y_test = torch.sin(x_test)
plt.plot(x_test.numpy(), y_test.numpy(), '.', label='TEST')
plt.legend();

Wrap the tensors into Dataset objects to simplify passing them around:

In [ ]:
xy_train = torch.utils.data.TensorDataset(x_train, y_train)
xy_test = torch.utils.data.TensorDataset(x_test, y_test)

Create (and save) the baseline model to use:

In [ ]:
torch.manual_seed(123)
baseline = torch.nn.Sequential(
    torch.nn.Linear(1, 20),
    torch.nn.ReLU(),
    torch.nn.Linear(20, 25),
    torch.nn.ReLU(),
    torch.nn.Linear(25, 1),
)
torch.save(baseline.state_dict(), 'baseline.pth')

Now, your job: implement the function below to build a train and test loop using hyperparameters specified as function arguments. Your code should produce a single scatter plot showing the TRAIN and TEST loss curves, similar to the ones we produced earlier.

In [ ]:
def learn(train_data, test_data, model, loss_fn=torch.nn.MSELoss(), n_epochs=200,
          batch_size=200, learning_rate=0.1, momentum=0.9, make_plot=True):
    """Perform a train and test loop with specified hyperparameters.
    
    Uses SGD optimization and produces a scatter plot of TRAIN and TEST
    loss versus epoch on a log-linear scale.
    
    Parameters
    ----------
    train_data : torch.utils.data.Dataset
        Container for the training input and target tensors to use.
    test_data : torch.utils.data.Dataset
        Container for the test input and target tensors to use.
    model : torch.nn.Module
        Neural network model of the data whose parameters will be learned.
    loss_fn : callable
        Function of (y_out, y_tgt) that calculates the scalar loss to use.
        Must support backwards() method.
    n_epochs : int
        Number of epochs of training to perform.
    batch_size : int
        Size of each (randomly shuffled) minibatch to use.
    learning_rate : float
        Learning rate to use for the SGD optimizer.
    momentum : float
        Momentum to use for the SGD optimizer.
        
    Returns
    -------
    tuple
        Tuple (train, test) of arrays of loss values after each epoch for
        the TRAIN and TEST samples, respectively. Both lists should have
        length equal to n_epochs.    
    """
    
    losses_train, losses_test = [], []
    loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    x_test, y_test = test_data.tensors

    # YOUR CODE HERE...  replace the next line with your solution
    
    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=momentum) #Create SDG optimizer
    
    for epoch in range(n_epochs): #Start training
        model.train()
        for x_batch, y_batch in loader:
            y_pred = model(x_batch)
            loss = loss_fn(y_pred,y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        losses_train.append(loss.data)
        
        model.eval() #Evaluate test data using trained network
        y_pred = model(x_test)
        loss = loss_fn(y_pred,y_test)
        losses_test.append(loss.data)
        
    if make_plot:
        plt.plot(losses_train, '.', label='TRAIN')
        plt.plot(losses_test, '.', label='TEST')
        plt.legend()
        plt.xlabel('Training Epoch')
        plt.ylabel('Loss')
        plt.yscale('log');
    return losses_train, losses_test


Test your code with the default hyperparameters using:

In [ ]:
# A correct solution should pass these tests.
torch.manual_seed(123)
baseline.load_state_dict(torch.load('baseline.pth'))
train, test = learn(xy_train, xy_test, baseline)
assert train[0] > 0.1 and test[0] > 0.1
assert train[60] < 1e-3 and test[60] < 1e-3
assert train[-1] < 1e-4 and test[-1] < 1e-4

Run the cell below to establish the same initial state (seed and model parameters) and repeat this learning loop with the optimizer momentum set to zero, to show its effect on the loss curves: large synchronized oscillations in both the TRAIN and TEST data.

In [ ]:
torch.manual_seed(123)
baseline.load_state_dict(torch.load('baseline.pth'))
learn(xy_train, xy_test, baseline, momentum=0);

Now we'll scan different values of the batch size, learning rate, and momentum to find good configurations.  In general it's best to do this by sampling random values of each parameter on any given trial, but for ease of interpretation we'll do it in 1-dimension for a few different sets of configurations.

Make plots showing the loss curves for the training data for each of the following sets of configurations:
* Learning rate = 0.06, batch size=51, for momenta between 0.79 and 1.0 in steps of 0.05
* Momentum=0.9, batch size=51, for learning rates between 0.01 and 0.22 in steps of 0.05
* Learning rate = 0.1, momentum=0.9, between 1 and the size of the training set, in steps of 100

Some notes:
* Make sure to reset the random seed and reload the network before each call to the `learn` function!

In [ ]:
# checking the momentum
momentum = np.arange(0.79,1.0,0.05)
for momenta in momentum: 
    #Run using momenta value
    torch.manual_seed(123)
    baseline.load_state_dict(torch.load('baseline.pth'))
    train, test = learn(xy_train, xy_test, baseline, batch_size=51, learning_rate=0.06, momentum=momenta, make_plot=False)
    
    plt.plot(train, '.', label='TRAIN p={}'.format(round(momenta,2)))
    plt.plot(test, '.', label='TEST p={}'.format(round(momenta,2)))
    plt.title("Changing Momentum")
    plt.legend()
    plt.xlabel('Training Epoch')
    plt.ylabel('Loss')
    plt.yscale('log')

    #Establish same initial state
    torch.manual_seed(123)
    baseline.load_state_dict(torch.load('baseline.pth'))
    learn(xy_train, xy_test, baseline, momentum=0,make_plot=False)

In [ ]:
# checking the learning rate
lr = np.arange(0.01,0.22,0.05)
for lr_val in lr:
    torch.manual_seed(123)
    baseline.load_state_dict(torch.load('baseline.pth'))
    train, test = learn(xy_train, xy_test, baseline, batch_size=51, learning_rate=lr_val, momentum=0.9, make_plot=False)
    
    plt.plot(train, '.', label='TRAIN lr={}'.format(round(lr_val,2)))
    plt.plot(test, '.', label='TEST lr={}'.format(round(lr_val,2)))
    plt.title("Changing Learning Rate")
    plt.legend()
    plt.xlabel('Training Epoch')
    plt.ylabel('Loss')
    plt.yscale('log')

    #Establish same initial state
    torch.manual_seed(123)
    baseline.load_state_dict(torch.load('baseline.pth'))
    learn(xy_train, xy_test, baseline, momentum=0,make_plot=False)

In [ ]:
# checking the batch size
bs = np.arange(100,1001,100)
for bs_val in bs:
    torch.manual_seed(123)
    baseline.load_state_dict(torch.load('baseline.pth'))
    train, test = learn(xy_train, xy_test, baseline, batch_size=int(bs_val), learning_rate=0.1, momentum=0.9, make_plot=False)
    
    plt.plot(train, '.', label='TRAIN bs={}'.format(round(bs_val,2)))
    plt.plot(test, '.', label='TEST bs={}'.format(round(bs_val,2)))
    plt.title("Changing Batch Size")
    plt.legend()
    plt.xlabel('Training Epoch')
    plt.ylabel('Loss')
    plt.yscale('log')

    #Establish same initial state
    torch.manual_seed(123)
    baseline.load_state_dict(torch.load('baseline.pth'))
    learn(xy_train, xy_test, baseline, momentum=0,make_plot=False)

## Problem 2

Let's play around with convolutional kernels and see if we can make some simple filters to apply to real images.  We'll use the `torchvision` package to transform common image formats:

In [ ]:
from PIL import Image
import torchvision

The pytorch `Conv2D` function that we used to implement some convolutional layers is a high-level function call, and manages the kernel weights internally.

In [ ]:
# With square kernels and equal stride
m = torch.nn.Conv2d(16, 33, 3, stride=2)

# non-square kernels and unequal stride and with padding
m = torch.nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))

# non-square kernels and unequal stride and with padding and dilation
m = torch.nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))

# generate some random input:
input = torch.randn(20, 16, 50, 100)

# compute the output:
output = m(input)

We can use the lower-level function `torch.nn.functional.conv2d` to manipulate kernels and tensors directly.  

Let's do something simple: let's define a 1-layer 5x5 matrix and convolve it with a 3x3 kernel to produce a 3x3 output feature map.  We'll make the kernel super-simple, the equivalent of an identity matrix for convolutions.

In [ ]:
channels = 1
h, w = 5, 5
x = torch.randn(1, channels, h, w)
print(x)
weights = torch.tensor([[0., 0., 0.],
                        [0., 1., 0.],
                        [0., 0., 0.]])
weights = weights.view(1, 1, 3, 3)

output = torch.nn.functional.conv2d(x, weights)
print(output)

If we add in some padding, then we can make the output tensor the same size as the input:

In [ ]:
output = torch.nn.functional.conv2d(x, weights,padding=1)
print(output)

In [ ]:
weights.shape

Of course it's possible to do this for more than just one channel, but that gets a bit tricky, so I'll leave that for you to play with if you want!

For now let's just try to do something simple with a one-channel image:

In [ ]:
import urllib.request
urllib.request.urlretrieve("http://mhance.scipp.ucsc.edu/banana-slug-misty-morehead.jpg", "banana_slug.jpg")

In [ ]:
img = Image.open("banana_slug.jpg")
plt.imshow(img)

We can turn this into a numpy array relatively easily:

In [ ]:
npimg=np.asarray(img)
plt.imshow(npimg)

Or we can use `torchvision` to turn it into a pytorch tensor, and then check that the tensor gives us back the image:

In [ ]:
imgtensor=torchvision.io.read_image("banana_slug.jpg")
plt.imshow(torchvision.transforms.functional.to_pil_image(imgtensor))

Let's make this a grayscale image so we only need to deal with a single channel:

In [ ]:
grayscaleimage=torchvision.transforms.Grayscale()(img)
grayscaleimage.show()

Note that if we try to show this image using `plt.imshow` it will try to color it in for us:

In [ ]:
plt.imshow(grayscaleimage)

So force it into grayscale:

In [ ]:
plt.imshow(grayscaleimage,cmap='gray')

We can make this into a tensor for processing:

In [ ]:
grayscaleimagetensor=torchvision.transforms.functional.to_tensor(grayscaleimage)

In [ ]:
grayscaleimagetensor=grayscaleimagetensor.view(1,1,674,900)

Now, your job: 
* implement a 3x3 kernel that shifts every pixel one step to the right
* take the difference between the resulting feature map and the original image
* display the resulting image

In [ ]:
# Your code here
weights = torch.tensor([[0., 0., 0.], #3x3 kernel that shift every pixel one step to the right
                        [1., 0., 0.],
                        [0., 0., 0.]])
weights = weights.view(1,1,3,3) #.view to reshape tensor?

output = torch.nn.functional.conv2d(grayscaleimagetensor,weights,padding=1) #Shift every pixel to the right

#diff = output-grayscaleimagetensor #Take the difference between resulting feature map and the original image
diff = grayscaleimagetensor-output
reduce_dim_diff = np.squeeze(diff) #Reduce dim(1,1,674,900) to dim(674,900)

#Print original image tensor
print("Original Image:")
print(grayscaleimagetensor)

#Print resulting feature map tensor
print("Resulting Feature Map:")
print(output)

#Print difference tensor
print("Difference:")
print(diff)

#Display resulting feature map
plt.figure()
plt.imshow(np.squeeze(output),cmap="gray")
plt.title("Resulting Feature Map")

#Display difference between resulting feature map and the original image
plt.figure()
plt.imshow(reduce_dim_diff,cmap='gray')
plt.title("Difference Between Resulting and Original Map")

## Problem 3

This is the problem where you should explore the input dataset that you'll use for your final project.

I'll do this for a mock dataset, but you should go through a similar exercise with your dataset.

I'm importing data that's been stored in the HDF5 format, using the [h5py](https://docs.h5py.org/en/stable/) python package:

In [ ]:
import h5py

If your data are stored in a different format, then certainly some of the details below will have to change, that's OK!  There are lots of ways to read in CSV files and more complicated file formats into python data structures.

If you're using ROOT files (common in high-energy physics), there are also lots of ways to get information from those files into python lists.  If you want to convert a ROOT file into HDF5, you can use [this script](https://github.com/scipp-atlas/mario-mapyde/blob/main/scripts/root2hdf5.py).

In [ ]:
signalfile='lowlevelAna_stops.hf5'
urllib.request.urlretrieve("http://mhance.scipp.ucsc.edu/%s" % signalfile, signalfile)

backgrfile='lowlevelAna_ttbar.hf5'
urllib.request.urlretrieve("http://mhance.scipp.ucsc.edu/%s" % backgrfile, backgrfile)

Let's inspect the contents of the HDF5 file just to get a sense for what's there:

In [ ]:
with h5py.File(signalfile, 'r') as hdf5file:
    print("Here are the keys in this file")
    print(hdf5file.keys())
    print("\n")
    
    # let's access the first key by its index:
    group=hdf5file[list(hdf5file.keys())[0]]
    print("What type does this key have?")
    print(type(group))
    print("It's a group, so we'll need to look inside of the group to find the dataset")
    print(group.keys())
    data=group["lowleveltree"]
    print("\n")
    
    print("These are the fields (features) of the dataset:")
    print(data.dtype.names)
    print("\n")
    
    print("Here are the contents of the first event:")
    print(data[0])
    print("\n")
    
    print("This is the number of jets (numjet) in all events: ")
    print(data["numjet"])
    print("\n")

Now that we know the format, accessing the data is a bit easier...  let's restrict the list of fields (I'll call them 'branches') we read in:

In [ ]:
branches=("numjet","numlepton","numbtagjet",
          "met","metphi",
          "lepton1pT","lepton1eta","lepton1phi",
          "lepton2pT","lepton2eta","lepton2phi",
          "jet1pT", "jet1eta", "jet1phi","jet1b",
          "jet2pT", "jet2eta", "jet2phi","jet2b",
          "jet3pT", "jet3eta", "jet3phi","jet3b",
          "jet4pT", "jet4eta", "jet4phi","jet4b",
          "jet5pT", "jet5eta", "jet5phi","jet5b",
          "jet6pT", "jet6eta", "jet6phi","jet6b")

with h5py.File(signalfile, 'r') as hdf5file:
    data=hdf5file[list(hdf5file.keys())[0]]["lowleveltree"]
    num_signal_events=len(data["numjet"])
    alldata=data[branches]

with h5py.File(backgrfile,'r') as hdf5file:
    data=hdf5file[list(hdf5file.keys())[0]]["lowleveltree"]
    num_backgr_events=len(data["numjet"])
    alldata = np.concatenate((alldata,data[branches]))

We read in the data as fields with a custom format, which is useful for keeping track of what's what, but 
let's store this as python lists instead

In [ ]:
Alldata=[[float(i) for i in j] for j in alldata]
print(Alldata[0])

The `Alldata` variable now contains all of the data for both signal and background events.  Since this will be for a binary classification problem, we want to keep track of which event is which, so we'll construct a `y` list that has 0's and 1's corresponding to whether each event is background (0) or signal (1).

In [ ]:
y = np.concatenate((np.ones (num_signal_events), 
                    np.zeros(num_backgr_events)))

Now we can split our full dataset into training and test samples.  However, remember that the data aren't randomized...  the first N events of `Alldata` are all signal events, and the rest of the events are background events.  So we need to shuffle the data first, then pull out training and test samples of a specific size.  The `scikit learn` toolkit has a handy function that does this for us:

In [ ]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(Alldata, y, test_size=1000, random_state=123)

We can look at the input features for our training dataset.  First let's make a map of the feature name to the feature data:

In [ ]:
X_train_before_scaling={}
for b in branches:
    X_train_before_scaling[b]=[event[branches.index(b)] for event in X_train]

Now plot the data for each feature

In [ ]:
fig = plt.figure(figsize=(20,30))
fig.tight_layout()
for b in range(len(branches)):
    ax=fig.add_subplot(9,4,1+b if b<8 else 2+b)
    plt.subplots_adjust(hspace=0.3,wspace=0.5)
    ax.hist(X_train_before_scaling[branches[b]])
    ax.set_xlabel(branches[b])
    ax.set_ylabel("Events/Bin")

This looks good, but the inputs are all over the place!  Some are strictly positive, some have values in the 1's or 10's while others have values in the 1000's.  This broad range of inputs will likely confuse our network and cause some features to have inappropriate influence on the results.  So we should scale the events for each feature to give a distribution that has mean=0 and variance=1.  We can do this for each feature with the following mapping:

$$x \rightarrow z=\frac{x-\mu}{\sigma}$$

where $x$ is the original value, $z$ is the 'scaled' value, $\mu$ is the mean of all values of $x$, and $\sigma$ is the standard deviation of $x$.  This isn't hard to do by hand, but again `scikit learn` provides a handy way to do this:

In [ ]:
# now scale based on the training data:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

Let's make that handy map for the scaled data so we can look at the data by feature:

In [ ]:
X_train_after_scaling={}
for b in branches:
    X_train_after_scaling[b]=[event[branches.index(b)] for event in X_train]

Print some values just to see what happens to a typical variable:

In [ ]:
print(X_train_before_scaling["met"][:5])
print(X_train_after_scaling["met"][:5])

Now re-draw the features after they've been scaled to see what they look like:

In [ ]:
fig = plt.figure(figsize=(20,30))
fig.tight_layout()
for b in range(len(branches)):
    ax=fig.add_subplot(9,4,1+b if b<8 else 2+b)
    plt.subplots_adjust(hspace=0.3,wspace=0.5)
    ax.hist(X_train_after_scaling[branches[b]])
    ax.set_xlabel(branches[b])
    ax.set_ylabel("Events/Bin")

These look a lot better!

One last thing: we've scaled our training data, but not our test data.  Fortunately the `sc` object we created above will remember the transformation that we applied to the training data, so we can transform the test data in exactly the same way (note this is just `transform`, not `fit_transform`):

In [ ]:
X_test = sc.transform (X_test)

Now do this for your data.  Try to get as far as you can.  Depending on the type of data you're analyzing, there may not be anywhere near as many plots to make as what I have above -- that's OK!  The important thing will be to identify what your features are, and make sure that the feature data are scaled appropriately.

### Display DataFrame

In [ ]:
# Your code here
import pandas as pd
# Google Drive link incase Github link doesn't work
GDrive_Link = "https://drive.google.com/file/d/1ZJP3osriK0isJDyBlyqIhbZ1gvT2N4aH/view?usp=sharing"
print("Google Drive link incase GitHub link doesn't work:")
print(GDrive_Link)

# Import CSV using raw Github link
Github_Link = "https://raw.githubusercontent.com/khangnngo12/Kinematics-of-M33/main/Redshift.csv"
df = pd.read_csv(Github_Link)
print("CSV file containing velocities (km/s), RA, Dec, and other informations:")
df
#raise NotImplementedError()

### Display Velocity Map

This section will display the velocity map with a fitted ellipse and two parabolas. The data points are created using RA and Dec, color-coded using its measured H-alpha velocities. The parabolas will divide the map into three areas: North, South, and Center. 

In [ ]:
center_RA = (1 + (33/60) + (51.75/3600))*15 #in deg
center_Dec = (30 + (39/60) + (36.630/3600)) # in deg

In [ ]:
def linear_func(x,m,b): #y=mx+b function
    return (m*x)+b

In [ ]:
def classification_func(RA,Dec):
    
    """
    Function to classified each point into North, South, and Center.
    
    Parameters
    ----------
    RA: ndarray,req
        Containing RA values used in map
    Dec: ndarray, req
        Containing Dec values used in map
        
    Returns
    ----------
    NorthParabola: ndarray containing bools. True == Inside North parabola
    SouthParabola: ndarray containing bools. True == Inside South parabola
    Center: ndarray containing bools. True == Inside Center region
    """
    
    import cmath
    theta = np.radians(-32)
    
    Dec_Val_NP_List = [] #list containing Dec values along North parabola
    Dec_Val_SP_List = [] #list containing Dec values along South parabola
    
    for RA_Val in RA:
    
        # find a,b,c for quadratic formula
        a_NorthPara = -0.5*np.sin(theta)
        b_NorthPara = 0.5*np.cos(theta)
        c_NorthPara = NorthPara_XCoor - RA_Val
        
        a_SouthPara = 0.5*np.sin(theta)
        b_SouthPara = -0.5*np.cos(theta)
        c_SouthPara = SouthPara_XCoor - RA_Val
        
        # calculate the discriminan
        d_NorthPara = (b_NorthPara**2) - (4*a_NorthPara*c_NorthPara) 
        d_SouthPara = (b_SouthPara**2) - (4*a_SouthPara*c_SouthPara)
        
        t_NorthPara = (-b_NorthPara + cmath.sqrt(d_NorthPara))/(2*a_NorthPara) # find t for North Parabola (Notice the plus)
        t_SouthPara = (-b_SouthPara - cmath.sqrt(d_SouthPara))/(2*a_SouthPara) # find t for South Parabola (Notice the subtract)
        
        Dec_Val_NorthPara = 0.5*(t_NorthPara*np.sin(theta) + (t_NorthPara**2)*np.cos(theta)) + NorthPara_YCoor #find corresponding Dec value using t
        Dec_Val_NP_List.append(Dec_Val_NorthPara)
        
        Dec_Val_SouthPara = -0.5*(t_SouthPara*np.sin(theta) + (t_SouthPara**2)*np.cos(theta)) + SouthPara_YCoor
        Dec_Val_SP_List.append(Dec_Val_SouthPara)
        
    NorthParabola = np.array(Dec-np.array(Dec_Val_NP_List) > 0)
    SouthParabola = np.array(Dec-np.array(Dec_Val_SP_List) < 0)
    Center = []
    for index in range(len(RA)):
        if NorthParabola[index] == False:
            if SouthParabola[index] == False:
                Center.append(True)
            else:
                Center.append(False)
        else:
            Center.append(False)
    
    return NorthParabola,SouthParabola,np.array(Center)

In [ ]:
import math
from math import pi, cos, sin

# Get datapoints from CSV files. 
df_qop2_sorted = df["QOP"] >= 2

fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.set_title("M33 Velocity Color Map (QOP >= 2)")
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,30.9)
ax.invert_xaxis()
ax.set_aspect('equal')
cc_map = ax.scatter(df["RA"][df_qop2_sorted]
                    ,df["DEC"][df_qop2_sorted]
                    ,c=df["Velocity (km/s)"][df_qop2_sorted],s=7,cmap='magma')
plt.colorbar(cc_map,label="Velocity (km/s)")
fig.set_figwidth(5)
fig.set_figheight(5)
plt.grid(True,linestyle="--")

#------------------------------

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

fig.set_figwidth(5)
fig.set_figheight(5)

The codes below will classified each datapoint into one of the three regions. After it is all sorted, is plotted again but this time color-coded based on which region it belong to. 

In [ ]:
# Call classification function to sort each point.
NorthParabola,SouthParabola,Center = classification_func(df["RA"],df["DEC"])

In [ ]:
import math
from math import pi, cos, sin

#Display Data
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.set_title("Color Coded Map")
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,31)
#ax.set_xlim(23,24)
#ax.set_ylim(30.2,31.2)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect("equal")
plt.grid(True,linestyle="--")

# Color code each regions
ax.scatter(df["RA"][NorthParabola],df["DEC"][NorthParabola],s=7,c="red",label="North")
ax.scatter(df["RA"][SouthParabola],df["DEC"][SouthParabola],s=7,c="blue",label="South")
ax.scatter(df["RA"][Center],df["DEC"][Center],s=7,c="green",label="Center")
plt.legend()

#-------------------------------------

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Fit Two Parabolas
theta = np.radians(-32) #degrees to tilt parabolas
t = np.linspace(-10,10,1000)
#x = (0.5*t)*np.cos(theta) - (t**2)*np.sin(theta) + x_2
#y = (0.5*t)*np.sin(theta) + (t**2)*np.cos(theta) + y_2
x_NorthPara = 0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + NorthPara_XCoor #x values of North parabola
y_NorthPara = 0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + NorthPara_YCoor #y values of North parabola
plt.plot(x_NorthPara,y_NorthPara,c="blue",linewidth=1,linestyle="--")
x_SouthPara = -0.5*(t*np.cos(theta) - (t**2)*np.sin(theta)) + SouthPara_XCoor #x values of South parabola
y_SouthPara = -0.5*(t*np.sin(theta) + (t**2)*np.cos(theta)) + SouthPara_YCoor #y values of South parabola
plt.plot(x_SouthPara,y_SouthPara,c="blue",linewidth=1,linestyle="--")

fig.set_figwidth(5)
fig.set_figheight(5)

### Histograms Using Velocity

In [ ]:
fig,ax = plt.subplots(3)
ax[0].hist(df["Velocity (km/s)"][NorthParabola])
ax[0].set_xlabel("Velocity (km/s)")

ax[1].hist(df["Velocity (km/s)"][SouthParabola])
ax[1].set_xlabel("Velocity (km/s)")

ax[2].hist(df["Velocity (km/s)"][Center])
ax[2].set_xlabel("Velocity (km/s)")

fig.tight_layout()

### Representing N,S,C as 0,1,2

In [ ]:
def y_train():
    
    """
    Assign each datapoint a value based on its location
    0 = North, 1 = Center, 2 = South
    
    Returns
    ----------
    y_train: ndarray containing 0,1,2 corresponing to zones.
    """
    
    y_train = []
    for index in np.arange(len(df["Velocity (km/s)"])):
        if index in np.where(NorthParabola==True)[0]:
            y_train.append(0)
        elif index in np.where(SouthParabola==True)[0]:
            y_train.append(2)
        else:
            y_train.append(1)
    
    return np.array(y_train)

In [ ]:
y_train = y_train() 

In [ ]:
print('Display TRUE results used to train the program:')
print(*y_train)

### Possible X-Train, X-Test, Y-Test? 

I'm still unsure about what I can use as the x-train, x-test, and y-test. I'm just trying out different possibilities here. 

One possible x-train I'm thinking about is to select two Dec values and classified each datapoint based on those Dec values then shuffle it. Then train it using y-train.

In [ ]:
import random
lim_0 = 30.5 #1st limit
lim_1 = 30.7 #2nd limit
x_train = []
for Dec in df["DEC"]:
    if Dec > lim_1:
        x_train.append(0)
    elif Dec < lim_0:
        x_train.append(2)
    else:
        x_train.append(1)
random.shuffle(x_train)

In [ ]:
import math
from math import pi, cos, sin

#Display Data
fig,ax = plt.subplots(1)
ax.set_xlabel("Right Ascension (deg)")
ax.set_ylabel("Declination (deg)")
ax.set_title("Randomized")
ax.set_xlim(23.2,23.8)
ax.set_ylim(30.15,31)
#ax.set_xlim(23,24)
#ax.set_ylim(30.2,31.2)
ax.invert_xaxis()
#ax.set_aspect(1/0.861)
ax.set_aspect("equal")
plt.grid(True,linestyle="--")

# Color code each regions
ax.scatter(df["RA"][np.where(np.array(x_train)==0)[0]],df["DEC"][np.where(np.array(x_train)==0)[0]],s=7,c="red",label="North")
ax.scatter(df["RA"][np.where(np.array(x_train)==1)[0]],df["DEC"][np.where(np.array(x_train)==1)[0]],s=7,c="blue",label="South")
ax.scatter(df["RA"][np.where(np.array(x_train)==2)[0]],df["DEC"][np.where(np.array(x_train)==2)[0]],s=7,c="green",label="Center")
plt.legend()

#-------------------------------------

#Fit Ellipse
u= center_RA       #x-position of the center
v= center_Dec    #y-position of the center
a= 0.52     #radius on the x-axis
b= 0.32      #radius on the y-axis
t_rot=math.radians(58) #rotation angle

t = np.linspace(0, 2*pi, 100)
Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
     #u,v removed to keep the same center location
R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
     #2-D rotation matrix

Ell_rot = np.zeros((2,Ell.shape[1]))
for i in range(Ell.shape[1]):
    Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

plt.plot(u+Ell_rot[0,:] , v+Ell_rot[1,:],c='black',linewidth=0.8)    #rotated ellipse

#Semi-minor Axis
x_SMi_0 = center_RA + (b*cos(np.radians(32))) #x-coor of left Semi-minor axis point 
y_SMi_0 = center_Dec - (b*sin(np.radians(32))) #y-coor of left Semi-minor axis point
x_SMi_1 = center_RA - (b*cos(np.radians(32))) #x-coor of right Semi-minor axis point
y_SMi_1 = center_Dec + (b*sin(np.radians(32))) #y-coor of right Semi-minor axis point
plt.scatter(x_SMi_0,y_SMi_0,s=15,c="black")
plt.scatter(x_SMi_1,y_SMi_1,s=15,c="black")

m_SMi = (y_SMi_1-y_SMi_0)/(x_SMi_1-x_SMi_0) #slope of line through Semi-minor axis 
b_SMi = y_SMi_0 - (m_SMi*x_SMi_0) #y-inter of line through Semi-minor axis
x_SMi = np.linspace(x_SMi_1,x_SMi_0,1000) #x values of line through Semi-minor axis
y_SMi = linear_func(x_SMi,m_SMi,b_SMi) #y values of line through Semi-minor axis
plt.plot(x_SMi,y_SMi,c='black',linewidth=0.8,linestyle="--")

#Semi-major Axis
x_SouthSMa = center_RA - (a*np.sin(np.radians(32))) #x-coor of North Semi-major axis point
y_SouthSMa = center_Dec - (a*np.cos(np.radians(32))) #y-coor of Nouth Semi-major axis point
x_NorthSMa = center_RA + (a*np.sin(np.radians(32))) #x-coor of South Semi-major axis point
y_NorthSMa = center_Dec + (a*np.cos(np.radians(32))) #y-coor of South Semi-major axis point
plt.scatter(x_NorthSMa,y_NorthSMa,c="black",s=15)
plt.scatter(x_SouthSMa,y_SouthSMa,c="black",s=15)

m_SMa = (y_NorthSMa-y_SouthSMa)/(x_NorthSMa-x_SouthSMa) #slope of line through Semi-major axis
b_SMa = y_SouthSMa - (m_SMa*x_SouthSMa) #y-inter of line through Semi-major axis 
x_SMa = np.linspace(x_NorthSMa,x_SouthSMa,1000) #x values of line through Semi-major axis
y_SMa = linear_func(x_SMa,m_SMa,b_SMa) #y values of line through Semi-major axis
plt.plot(x_SMa,y_SMa,c='black',linewidth=0.8,linestyle="--")

NorthPara_XCoor = x_NorthSMa - ((19*a/20)*np.sin(np.radians(32))) #coordinates to move North parabola to
NorthPara_YCoor = linear_func(NorthPara_XCoor,m_SMa,b_SMa)

SouthPara_XCoor = x_SouthSMa + ((19*a/20)*np.sin(np.radians(32))) #coordinate to move South parabola to
SouthPara_YCoor = linear_func(SouthPara_XCoor,m_SMa,b_SMa)

#Define two limits
plt.hlines(30.5,23.2,23.8,linestyle="--",linewidth=1)
plt.hlines(30.7,23.2,23.8,linestyle="--",linewidth=1)

fig.set_figwidth(5)
fig.set_figheight(5)

Would I just take a subset of x_train and y_train and use them as x_test and y_test? 

In [ ]:
x_test = x_train[800:1000] #Possible x_test?
y_test = y_train[800:1000] #Possible y_test?